# EEG Feature Extraction


## Importing all required libraries


In [1]:
import numpy as np
import pandas as pd
import os
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal
import librosa as lr
import librosa.feature as lrf
import sklearn as sk
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal
import librosa as lr
import librosa.feature as lrf
from scipy.signal import welch
import pywt
from pywt import *
from scipy.signal import periodogram
#from pyemd import emd
from scipy.signal import hilbert
from scipy.stats import linregress, skew, kurtosis
from scipy.fft import fft, fftfreq

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import nolds
from pyentrp import entropy as ent
from scipy.signal import detrend
from nolds import dfa

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
def hjorth_parameters(eeg_signal):
    # Calculate the first derivative (slope) of the EEG signal
    diff_signal = np.diff(eeg_signal)
    
    # Calculate the variance (activity) of the original signal
    activity = np.var(eeg_signal)
    
    # Calculate the variance (activity) of the first derivative (slope)
    mobility = np.var(diff_signal)
    
    # Calculate the mobility parameter (square root of mobility divided by activity)
    mobility /= activity
    
    # Calculate the second derivative of the EEG signal
    diff2_signal = np.diff(diff_signal)
    
    # Calculate the complexity (square root of the mobility divided by the mobility of the first derivative)
    complexity = np.sqrt(mobility / (np.var(diff2_signal) / activity))
    
    return activity, mobility, complexity

In [3]:
sr = 128
n_mfcc = 10 
n_chr = 20
n_mel = 15
n_tonnetz = 15
frequency_bands = {
    'gamma': (30,64),
    'beta': (13, 30),
    'alpha': (8, 13),
    'theta': (4, 8),
    'delta': (1, 4),
    }
sampling_frequency = 128
# Number of sample points
N = sr*3
# sample spacing
T = 1.0 / sr

In [4]:
def feature_extraction(signal):

    feature_vector = {}
    
    signal -= np.mean(signal)
    
    # 0 indices are due to array shape
    feature_vector['spc_cnt'] = lrf.spectral_centroid(y=signal, sr=sr)[0][0] # Spectral Centroid
    feature_vector['spc_roff'] = lrf.spectral_rolloff(y=signal, sr=sr)[0][0] # Rolloff
    feature_vector['zc']  = np.array(np.sum(np.abs(np.diff(np.sign(signal)))) / (2 * len(signal)))
        
    for idx, mfcc in enumerate(lrf.mfcc(y=signal, n_mfcc=n_mfcc, sr=sr)): # First 5 MFCCs
        feature_vector['mfcc_' + str(idx)] = mfcc[0]
    
    for idx, chroma in enumerate(lrf.chroma_stft(y=signal, n_chroma=n_chr, sr=sr)): #chromagram
        feature_vector['chr_' + str(idx)] = chroma[0]

    for idx, mel in enumerate(lr.power_to_db(lrf.melspectrogram(y=signal, sr=sr))[:n_mel, :]):
        feature_vector['mel_' + str(idx)] = mel[0]
    
    # Iterate over each frequency band
    band_powers = {}

    # Calculate the power spectral density (PSD) using Welch's method
    frequencies, psd = welch(signal, fs=sampling_frequency, nperseg=1024)

    # Iterate over each frequency band
    # iterations are reversed due to performance differences in certain models
    # TODO: reversing process should be improved, way too clunky rn.
    for band, (low_freq, high_freq) in reversed(frequency_bands.items()):
        # Find indices corresponding to the specified frequency range
        band_indices = np.where((frequencies >= low_freq) & (frequencies < high_freq))
        # Integrate PSD within the band's frequency range to compute band power
        band_power = np.trapz(psd[band_indices], frequencies[band_indices])
        band_powers[band] = band_power
        feature_vector[band + '_power'] = band_power
    
    for band in reversed(list(band_powers)):
        for child_band in reversed(list(band_powers)):
            if child_band == band:
                continue
            feature_vector[band + '_' + child_band] = band_powers[band]/ band_powers[child_band]
        band_powers.pop(band)
    
    # Calculate the first differences
    first_differences = np.diff(signal, n=1)

    # Calculate the mean of the absolute values of the first differences
    feature_vector['mean_abs_sec_dif'] = np.mean(np.abs(first_differences))

    feature_vector['dfa'] = dfa(signal, overlap=False)
    
    yf = fft(signal)
    yf = 2.0/N * np.abs(yf[0:N//2])
    np.clip(yf, 0, 15)
    yf = (yf - np.min(yf))/(np.max(yf) - np.min(yf))
    peaks, _ = scipy.signal.find_peaks(yf, height=0)
    peaks, _ = scipy.signal.find_peaks(yf, height=np.max(yf[peaks])*0.25)

    xf = fftfreq(N, T)[:N//2]

    # frequency of the maximum peak    
    #feature_vector['peak_freq'] = xf[yf == np.max(yf[peaks])]

    # maximum frequency of peaks
    #feature_vector['max_freq'] = xf[peaks][len(xf[peaks]) - 1]
    
    # peak slope
    res = linregress(xf[peaks], yf[peaks])
    feature_vector['slope'] = res.slope
    
    feature_vector['skew'] = [skew(signal)][0] #no
    feature_vector['kurtosis'] = [kurtosis(signal)][0] #no

    activity, mobility, complexity = hjorth_parameters(signal) #no
    feature_vector['activity'] = [activity][0]
    feature_vector['mobility'] = [mobility][0]
    feature_vector['complexity'] = [complexity][0]
    feature_vector['rms'] = np.sqrt(np.mean(signal**2))
    feature_vector['tempo'] = lr.beat.tempo(y=signal)[0]

    
    for idx, tonal in enumerate(lrf.tonnetz(y=signal)[:n_tonnetz, :]):
        feature_vector['ton_' + str(idx)] = tonal[0]
        
    return feature_vector
    
    # rejected features, discuss whether to include them in the paper or not.



    '''
    [cA5, cD5, cD4, cD3, cD2, cD1] = wavedec(signal, 'db1', level=5)
    coeffs = [cA5, cD5, cD4, cD3, cD2, cD1]
    db1 = pywt.Wavelet('db1')
    
    re0 = pywt.waverec([coeffs[0]], db1)
    re1 = pywt.waverec([coeffs[1]], db1)
    re2 = pywt.waverec([coeffs[2]], db1)
    re3 = pywt.waverec([coeffs[3]], db1)
    re4 = pywt.waverec([coeffs[4]], db1)
    re5 = pywt.waverec([coeffs[5]], db1)
    '''


    # hata veren featureler

    '''    
    
    # Rhythymic Content Features
    peaks = -np.sort(-lr.onset.onset_strength(signal, sr = 128))
    A0 = peaks[0]
    A1 = peaks[1]
    RA = A1/A0    
    
    extracted_features.append(np.mean(lrf.tempogram(signal)))
    extracted_features.append(A0)
    extracted_features.append(A1)
    extracted_features.append(RA)

    '''
    #feature_vector.append([lr.onset.onset_strength(signal)]) # Flux
    #feature_vector.append(lr.feature.zero_crossing_rate(signal)) # Zero Crossings

In [5]:
data_root = 'EEG Data/'
def get_data(filename):
    '''
    Returns the EEG value of a given .mat file.
    The value consists of 25 channels, but only 3-16 are EEG channels.
    
    returns: state, the eeg values with states seperated.
    '''
    mat = scipy.io.loadmat(os.path.join(data_root, filename))
    data = mat['o']['data'][0, 0]
    FS = mat['o']['sampFreq'][0][0][0][0]

    states = {
     'focused': data[:FS * 10 * 60, :],
      'unfocused': data[FS * 10 * 60:FS * 20 * 60, :],
      'drowsy': data[FS * 20 * 60:FS * 30 * 60, :]
    }
    
    return states

In [6]:
# brute processings
channel_indices = np.array(range(3, 17))
channel_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
channel_names = ['F3', 'F4', 'O1', 'O2', 'F7', 'F8', 'FC5', 'FC6']
channel_map = dict(zip(channel_names, channel_indices))

rows_list = []
from tqdm import tqdm
for exp_idx in range(1,35):
    if exp_idx == 28: # corrupt file
        continue
    states = get_data(f"eeg_record{exp_idx}.mat")
    for ch_name, ch_idx in tqdm(channel_map.items()):
        for state, eeg in states.items():
            for i in range((eeg.shape[0]//(128*3))):
                try:
                    data = eeg[i*(128*3):(i+1)*(128*3), ch_idx]
                    powers = feature_extraction(data)
                    #powers['channel'] = ch_name
                    powers['state'] = state
                    rows_list.append(powers)
                except ValueError:  #raised if `y` is empty.
                    pass

100%|██████████| 8/8 [05:39<00:00, 42.47s/it]


In [7]:
#df = pd.DataFrame.from_dict(rows_list)
df = pd.DataFrame.from_records(rows_list).fillna(0)

## Explore and export data

Save the features to CSV file

In [8]:
df.to_csv("archive.csv")

In [6]:
df = pd.read_csv('archive.csv')
df = df.drop('Unnamed: 0', axis=1)

In [7]:
def remove_state(df, state):
    df = df.drop(df[df['state'] == state].index)
    return df

In [8]:
removed_state = 'unfocused'
df = remove_state(df, removed_state)
df.loc[df['state'] == 'focused', 'state'] = '1'
df.loc[df['state'] == 'drowsy', 'state'] = '0'

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop('state', axis=1)
y = df.iloc[:, -1:]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=447)

# apply normalization after splitting to avoid leakage
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
import warnings

import numpy as np
import pandas as pd
import scipy.stats as stats
import sklearn as sk
import xgboost as xgb
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.exceptions import DataConversionWarning
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score,
                             classification_report, confusion_matrix, f1_score,
                             log_loss, precision_score, recall_score,
                             roc_auc_score, roc_curve)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import itertools
knn = KNeighborsClassifier(leaf_size= 10)
svm = SVC(C=10.0, kernel='rbf', gamma=0.1, random_state=1)
xgb = GradientBoostingClassifier(
            loss='log_loss', n_estimators=300, learning_rate=0.1, max_depth=10, random_state=1)
models = [knn, svm, xgb]

In [11]:
#X_train = np.array(X_train)
#y_train = np.array(y_train)
#X_test = np.array(X_test)
#y_test = np.array(y_test)

knn.fit(X_train, y_train)
training_acc = knn.score(X_train, y_train)
test_acc = knn.score(X_test, y_test)


In [12]:
for model in models:
    model.fit(X_train, y_train)
    training_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)

    print(training_acc)
    print(test_acc)
    

KeyboardInterrupt: 

In [1]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix 

param_grid = {'C': [10],  
              'gamma': [0.0359,0.0361,0.0363,0.0365,0.0367,0.0369,0.0371], 
              'kernel': ['rbf'],
              'degree': [3]}  

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
print('Result of the best model on the test set: ', grid_predictions)
print(grid.best_params_) 

NameError: name 'X_train' is not defined